In [79]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#activeslosses = pd.read_csv('../data/Avtice-losses.csv', skiprows = 1)
#activeslosses

In [80]:
activeslosses = pd.read_csv('../data/Avtice-losses.csv', skiprows = 1)
activeslosses['Zeitstempel'] = pd.to_datetime(activeslosses['Zeitstempel']) - pd.Timedelta(minutes=15)
activeslosses.set_index(activeslosses.columns[0], inplace=True)
activeslosses['MWh'] = activeslosses['kWh']/1000
activeslosses = activeslosses[~activeslosses.index.duplicated(keep='first')]
activeslosses = activeslosses.resample('15T').asfreq()


Forecast_renew = pd.read_csv('../data/Forecast-renewable-generation.csv', skiprows = 0)
Forecast_renew['datetime'] = pd.to_datetime(Forecast_renew['datetime'])
Forecast_renew.set_index(Forecast_renew.columns[0], inplace=True)
Forecast_renew = Forecast_renew[~Forecast_renew.index.duplicated(keep='first')]
Forecast_renew = Forecast_renew.resample('H').asfreq()


Forecast_temp = pd.read_csv('../data/Forecast-temperature.csv', skiprows = 0)
Forecast_temp['datetime'] = pd.to_datetime(Forecast_temp['datetime'])
Forecast_temp.set_index(Forecast_temp.columns[0], inplace=True)
Forecast_temp = Forecast_temp[~Forecast_temp.index.duplicated(keep='first')]


NTC = pd.read_csv('../data/NTC.csv', skiprows = 0)
NTC['datetime'] = pd.to_datetime(NTC['datetime'])
NTC.set_index(NTC.columns[0], inplace=True)
NTC = NTC[~NTC.index.duplicated(keep='first')]
NTC = NTC.resample('H').asfreq()


In [81]:
Forecast_renew

,solar_fore_de [MW],solar_fore_it [MW],wind_fore_de [MW],wind_fore_it [MW]
datetime,,,,
2019-01-01 00:00:00,0.0,0.0,21344.8514,4302.6977
2019-01-01 01:00:00,0.0,0.0,23052.3310,4596.5916
2019-01-01 02:00:00,0.0,0.0,24969.9701,4478.5564
2019-01-01 03:00:00,0.0,0.0,27082.9626,4323.3712
2019-01-01 04:00:00,0.0,0.0,26890.9717,4231.8283
...,...,...,...,...
2021-12-31 19:00:00,0.0,0.0,36997.7200,1108.4000
2021-12-31 20:00:00,0.0,0.0,35666.9300,1077.9700
2021-12-31 21:00:00,0.0,0.0,34383.8800,1048.2800


In [82]:
#data_preprocessing

activeslosses = activeslosses.interpolate(method='polynomial', order=2)

activeslosses_hour = activeslosses.resample('H').sum()



Forecast_renew = Forecast_renew.interpolate(method='polynomial', order=2)


Forecast_temp_hourly = Forecast_temp.resample('H').asfreq()

Forecast_temp_hourly_lin = Forecast_temp_hourly.interpolate(method='linear')

Forecast_temp_hourly_poly = Forecast_temp_hourly.interpolate(method='polynomial', order=2)

Forecast_temp_hourly_poly.loc[pd.to_datetime('2019-01-01 00:00:00')] = Forecast_temp_hourly_poly.loc['2019-01-01 01:00:00   ']




NTC = NTC.interpolate(method='polynomial', order=2)


"""
temp = Forecast_temp_hourly.loc['2019-01']
temp_lin = Forecast_temp_hourly_lin.loc['2019-01']
temp_poly = Forecast_temp_hourly_poly.loc['2019-01']

plt.plot(temp_lin.index, temp_lin['temperature_fore_ch'], 'o', label='Data')  # 'o-' ensures that points are marked with circles and lines connect them.
plt.plot(temp_poly.index, temp_poly['temperature_fore_ch'], 'o', label='Data')  # 'o-' ensures that points are marked with circles and lines connect them.
plt.plot(temp.index, temp['temperature_fore_ch'], 'o', label='Data')  # 'o-' ensures that points are marked with circles and lines connect them.

"""

"\ntemp = Forecast_temp_hourly.loc['2019-01']\ntemp_lin = Forecast_temp_hourly_lin.loc['2019-01']\ntemp_poly = Forecast_temp_hourly_poly.loc['2019-01']\n\nplt.plot(temp_lin.index, temp_lin['temperature_fore_ch'], 'o', label='Data')  # 'o-' ensures that points are marked with circles and lines connect them.\nplt.plot(temp_poly.index, temp_poly['temperature_fore_ch'], 'o', label='Data')  # 'o-' ensures that points are marked with circles and lines connect them.\nplt.plot(temp.index, temp['temperature_fore_ch'], 'o', label='Data')  # 'o-' ensures that points are marked with circles and lines connect them.\n\n"

In [88]:
#merge all

merged_df = pd.merge(activeslosses_hour, Forecast_renew, left_index=True, right_index=True, how='outer')
merged_df = pd.merge(merged_df, Forecast_temp_hourly_poly, left_index=True, right_index=True, how='outer')
merged_df = pd.merge(merged_df, NTC, left_index=True, right_index=True, how='outer')


merged_df.to_csv('merged_data.csv', index=True)

In [ ]:
merged_df

In [87]:
nan_columns = merged_df.columns[merged_df.isna().any()].tolist()
print(nan_columns)


[]


In [74]:
#print(activeslosses_hour.shape, Forecast_renew.shape, Forecast_temp_hourly_poly.shape, NTC.shape)

In [28]:
#activeslosses_hour.iloc[10872:10888]

In [75]:
#print(NTC[NTC.index.duplicated()])

In [76]:
"""
print(activeslosses[activeslosses.index.duplicated()])
activeslosses.loc['2019-10-27 01:45:00 ']
activeslosses.loc['2019-10-27 02:00:00']
activeslosses.loc['2019-10-27 01:45:00 ']
activeslosses.loc['2019-10-27 02:00:00']
"""

"\nprint(activeslosses[activeslosses.index.duplicated()])\nactiveslosses.loc['2019-10-27 01:45:00 ']\nactiveslosses.loc['2019-10-27 02:00:00']\nactiveslosses.loc['2019-10-27 01:45:00 ']\nactiveslosses.loc['2019-10-27 02:00:00']\n"

In [80]:
"""
print(activeslosses.describe())
print(Forecast_renew.describe())
print(Forecast_temp.describe())
print(NTC.describe())
"""


'\nprint(activeslosses.describe())\nprint(Forecast_renew.describe())\nprint(Forecast_temp.describe())\nprint(NTC.describe())\n'